# &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; Detect Pneumonia (2021)




## The code will be split in 5 main parts: <br>

[<li> 1. Libraries](#1) <br>

[<li> 2. Input](#2) <br>
&emsp;  &emsp;  &ensp; [*2.1 Hyperparameters*](#2.1) <br>
&emsp;  &emsp;  &ensp; [ *2.2 Data Directories*](#2.2) <br>
&emsp;  &emsp;  &ensp; [*2.3 Generalization Variables*](#2.3) <br>

[<li> 3. Data Preprocessing](#3) <br>
&emsp;  &emsp;  &ensp; [*3.1 Setting Variables*](#3.1) <br>
&emsp;  &emsp;  &ensp; [*3.2 Extra Data*](#3.2) <br>
&emsp;  &emsp;  &ensp; [*3.3 Image Read*](#3.3) <br>
&emsp;  &emsp;  &ensp; [*3.4 Cross Validation Split*](#3.4) <br>
&emsp;  &emsp;  &ensp; [*3.5 Data Preperation*](#3.5) <br>

[<li> 4. Model ](#4) <br>
&emsp;  &emsp;  &ensp; [*4.1 Callbacks*](#4.1) <br>
&emsp;  &emsp;  &ensp; [*4.2 Models* ](#4.2) <br>
&emsp;  &emsp;  &ensp; [*4.3 Model Fit*](#4.3) <br>
&emsp;  &emsp;  &ensp; [*4.4 Fine Tuning*](#4.4) <br>

[<li> 5. Predictions ](#5) <br>
&emsp;  &emsp;  &ensp; [ *5.1 Test Images*](#5.1) <br>
&emsp;  &emsp;  &ensp; [ *5.2 Export Predictions*](#5.2) <br>
&emsp;  &emsp;  &ensp; [ *5.3 Voting Ensemble*](#5.3) <br>

<a name="1"></a>
##  1. Libraries

In [ ]:
# Core
import os
import cv2
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# TF
from tensorflow import keras
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler

# Keras
from keras import optimizers, losses
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator


<a name="2"></a>
## 2. Input

<a name="2.1"></a>
### &ensp; *2.1 Hyperparameters*

In [ ]:
selected_model = 'efficientnet_B3'

Hyperparameters = {
    'efficientnet_B0':{
        'img_size':224,
        'epochs': 25,
        'batch_size':8,
        'learning_rate':1e-4,
        'fine_tune_epochs':25,
        'unfreeze_layers':-25
        },
    'efficientnet_B1':{
        'img_size':240,
        'epochs': 50,
        'batch_size':16,
        'learning_rate':1e-3,
        'fine_tune_epochs':80,
        'unfreeze_layers':-40
        },
    'efficientnet_B2':{
        'img_size':300,
        'epochs': 50,
        'batch_size':32,
        'learning_rate':1e-3,
        'fine_tune_epochs':80,
        'unfreeze_layers':-25
        },
    'efficientnet_B3':{
        'img_size':300,
        'epochs': 40,
        'batch_size':32,
        'learning_rate':1e-3,
        'fine_tune_epochs':50,
        'unfreeze_layers':-30
        },
    'efficientnet_B4':{
        'img_size':300,
        'epochs': 25,
        'batch_size':8,
        'learning_rate':1e-4,
        'fine_tune_epochs':25,
        'unfreeze_layers':-30
        },
    'efficientnet_B5':{
        'img_size':456,
        'epochs': 25,
        'batch_size':8,
        'learning_rate':1e-4,
        'fine_tune_epochs':25,
        'unfreeze_layers':-25
        }
}


<a name="2.2"></a>
### &ensp; *2.2 Data Directories* 

In [ ]:
# Data paths
train_path = '/Users/thxsg/OneDrive - International Hellenic University/1. IHU (2021)/5. Advanced Machine Learning/2. Homework/detect-pneumonia-fall-2021/train_images'
test_path = '/Users/thxsg/OneDrive - International Hellenic University/1. IHU (2021)/5. Advanced Machine Learning/2. Homework/detect-pneumonia-fall-2021/test_images'
label_csv = '/Users/thxsg/OneDrive - International Hellenic University/1. IHU (2021)/5. Advanced Machine Learning/2. Homework/detect-pneumonia-fall-2021/labels_train.csv'

# Checkpoints
path_c = '/Users/thxsg/Documents/Python Scripts/Checkpoints/'

# Submissions
sub = '/Users/thxsg/Documents/Python Scripts/Submissions/'

# Softmax output for the model
soft_single = '/Users/thxsg/Documents/Python Scripts/Soft-Votes.csv'

# Path to store softamax CSVs for Voting
soft_path = '/Users/thxsg/OneDrive - International Hellenic University/1. IHU (2021)/5. Advanced Machine Learning/2. Homework/Attempts/Softmaxes'


<a name="2.3"></a>
### &ensp; *2.3 Generalization Variables*

In [ ]:
colx = 'file_name'
coly = 'class_id'


<a name="3"></a>
## 3. Data Preprocessing 

<a name="3.1"></a>
### &ensp; *3.1 Setting Variables*

In [ ]:
# Hyperparameters
img_size = Hyperparameters[selected_model]['img_size']
epochs = Hyperparameters[selected_model]['epochs']
batch_size = Hyperparameters[selected_model]['batch_size']
learning_rate = Hyperparameters[selected_model]['learning_rate']
fine_tune_epochs = Hyperparameters[selected_model]['fine_tune_epochs']
unfreeze_layers = Hyperparameters[selected_model]['unfreeze_layers']

# Other Variables
labels = pd.read_csv(label_csv)
classes = labels[coly].unique()
num_cl = len(classes)


<a name="3.2"></a>
### &ensp; *3.2 Extra Data*

In [ ]:
# Round the max number of images of the biggest class to a multiplier of 10,
# for cross validation.
lengths = []
for i in range(0, num_cl):
   exec('lengths.append(len(labels.loc[labels[coly] == ' + str(i) + ']))')
maxlen = max(lengths)
rr = - maxlen%10
new_max = maxlen + rr

# Then calculate the numbers for each class needed for data augmentation,
# so each class has the same number of images.
for i in range(0, num_cl):
   exec('d' + str(i) + ' = new_max - len(labels.loc[labels[coly] == ' + str(i) + '])')

# Creating the list for the images that will be augmented.
exec('df_extra = labels.loc[labels[coly] == (num_cl - 1)].iloc[:d' + str(num_cl - 1) + ',][colx]')
for i in range(num_cl - 2, -1, -1):
  exec('df_extra = labels.loc[labels[coly] == ' + str(i) + '].iloc[:d' + str(i) + ',][colx].append(df_extra)')


<a name="3.3"></a>
### &ensp; *3.3 Image Read*

In [ ]:
# Read Images
x_train = []
for img in labels[colx]:
  img_array = cv2.imread(os.path.join(train_path, img))
  res_array = cv2.resize(img_array, (img_size, img_size))
  x_train.append(res_array)

x_train_extra = []
for img in df_extra:
  img_array = cv2.imread(os.path.join(train_path, img))
  res_array = cv2.resize(img_array, (img_size, img_size))
  x_train_extra.append(res_array)

# Transform Data to Tensorflow shape
x_train = np.array(x_train).reshape(-1, img_size, img_size, 3)
x_train_extra = np.array(x_train_extra).reshape(-1, img_size, img_size, 3)

# Create augmentation function for extra data
augm = keras.Sequential([layers.experimental.preprocessing.RandomZoom(0.05),])

# Image data
x_train = np.concatenate((x_train, augm(x_train_extra)))

# Image data labels
y_train = np.array(labels[coly])
for i in range(0, num_cl):
  exec('y_train = np.append(y_train, np.repeat(' + str(i) + ', d' + str(i) + '))')


<a name="3.4"></a>
### &ensp; *3.4 Cross Validation Split*

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    x_train,
    y_train,
    stratify = y_train,
    test_size = 0.1,
    random_state = 1)
    

<a name="3.5"></a>
### &ensp; *3.5 Data Preperation*

In [ ]:
# 1-hot-encoding, for categorical cross entropy
y_train = to_categorical(y_train, num_cl)
y_val = to_categorical(y_val, num_cl)

# Prepare batches for the model
train = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(batch_size)
val = tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(batch_size)


<a name="4"></a>
## 4. Model

<a name="4.1"></a>
### &ensp; *4.1 Callbacks*

In [ ]:
# Checkpoints
checkpoint_acc = tf.keras.callbacks.ModelCheckpoint(filepath = path_c + 'Acc_CP - ' + selected_model + '.h5',
                                                    monitor='val_acc',
                                                    save_best_only=True)

checkpoint_loss = tf.keras.callbacks.ModelCheckpoint(filepath = path_c + 'Loss_CP - ' + selected_model + '.h5',
                                                     monitor='val_loss',
                                                     save_best_only=True)

checkpoint_acc_ft = tf.keras.callbacks.ModelCheckpoint(filepath = path_c + 'Acc_CP - ' + selected_model + ' FT.h5',
                                                       monitor='val_acc',
                                                       save_best_only=True)

checkpoint_loss_ft = tf.keras.callbacks.ModelCheckpoint(filepath = path_c + 'Loss_CP - ' + selected_model + ' FT.h5',
                                                        monitor='val_loss',
                                                        save_best_only=True)

# Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(patience=13,
                                                     monitor='val_acc',
                                                     restore_best_weights=True)

# Learning Rate Modifiers
def lr_schedule(epoch):
    global epochs
    global learning_rate

    lr = learning_rate
    if epoch > 0.9*epochs:
        lr *= 0.5e-3
    elif epoch > 0.8*epochs:
        lr *= 1e-3
    elif epoch > 0.6*epochs:
        lr *= 1e-2
    elif epoch > 0.4*epochs:
        lr *= 1e-1
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=np.sqrt(0.1),
                               mode='min',
                               patience=3,
                               min_lr=0.5e-6,
                               verbose=1)


<a name="4.2"></a>
### &ensp; *4.2 Models*

In [ ]:
def efficientnet_B0():
        global num_cl
        global img_size
        
        base_model = tf.keras.applications.EfficientNetB0(include_top=False,
                                     weights='imagenet',
                                     input_shape=(img_size, img_size, 3))
        x = base_model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = Dense(512, activation=None, kernel_regularizer = L1L2(l1=1e-5, l2=1e-3))(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(0.3)(x)
        x = Dense(512, activation=None, kernel_regularizer = L1L2(l1=1e-5, l2=1e-3))(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(0.3)(x)
        out = Dense(num_cl, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=out)

        for layer in base_model.layers:
          layer.trainable = False

        return model


def efficientnet_B1():
        global num_cl
        global img_size

        base_model = tf.keras.applications.EfficientNetB1(include_top=False,
                                     weights='imagenet',
                                     input_shape=(img_size, img_size, 3))
        x = base_model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = Dense(512, activation=None, kernel_regularizer = L1L2(l1=1e-5, l2=1e-3))(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(0.3)(x)
        x = Dense(512, activation=None, kernel_regularizer = L1L2(l1=1e-5, l2=1e-3))(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(0.3)(x)
        out = Dense(num_cl, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=out)

        for layer in base_model.layers:
          layer.trainable = False

        return model


def efficientnet_B2():
        global num_cl
        global img_size

        base_model = tf.keras.applications.EfficientNetB2(include_top=False,
                                     weights='imagenet',
                                     input_shape=(img_size, img_size, 3))
        x = base_model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = Dense(512, activation=None, kernel_regularizer = L1L2(l1=1e-5, l2=1e-3))(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(0.3)(x)
        x = Dense(512, activation=None, kernel_regularizer = L1L2(l1=1e-5, l2=1e-3))(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(0.3)(x)
        out = Dense(num_cl, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=out)

        for layer in base_model.layers:
          layer.trainable = False

        return model


def efficientnet_B3():
        global num_cl
        global img_size

        base_model = tf.keras.applications.EfficientNetB3(include_top=False,
                                     weights='imagenet',
                                     input_shape=(img_size, img_size, 3))
        x = base_model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = Dense(512, activation=None, kernel_regularizer = L1L2(l1=1e-5, l2=1e-3))(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(0.3)(x)
        x = Dense(512, activation=None, kernel_regularizer = L1L2(l1=1e-5, l2=1e-3))(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(0.3)(x)
        out = Dense(num_cl, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=out)

        for layer in base_model.layers:
          layer.trainable = False

        return model


def efficientnet_B4():
        global num_cl
        global img_size

        base_model = tf.keras.applications.EfficientNetB4(include_top=False,
                                     weights='imagenet',
                                     input_shape=(img_size, img_size, 3))
        x = base_model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = Dense(512, activation=None, kernel_regularizer = L1L2(l1=1e-5, l2=1e-3))(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(0.3)(x)
        x = Dense(512, activation=None, kernel_regularizer = L1L2(l1=1e-5, l2=1e-3))(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(0.3)(x)
        out = Dense(num_cl, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=out)

        for layer in base_model.layers:
          layer.trainable = False

        return model


def efficientnet_B5():
        global num_cl
        global img_size

        base_model = tf.keras.applications.EfficientNetB5(include_top=False,
                                     weights='imagenet',
                                     input_shape=(img_size, img_size, 3))
        x = base_model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = Dense(512, activation=None, kernel_regularizer = L1L2(l1=1e-5, l2=1e-3))(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(0.3)(x)
        x = Dense(512, activation=None, kernel_regularizer = L1L2(l1=1e-5, l2=1e-3))(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(0.3)(x)
        out = Dense(num_cl, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=out)

        for layer in base_model.layers:
          layer.trainable = False

        return model


In [ ]:
exec('model = ' + selected_model + '()')
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate, decay = learning_rate * 0.1),
              metrics=['acc'])


<a name="4.3"></a>
### &ensp; *4.3 Model Fit* 

In [ ]:
hist = model.fit(train,
                 validation_data = val,
                 epochs = epochs,
                 steps_per_epoch = len(x_train) // batch_size,
                 callbacks=[checkpoint_acc, checkpoint_loss, lr_reducer, lr_scheduler])


In [ ]:
# Variable to mark whether the model is Fine Tuned in exported CSV name
fine_t = ''

# Load model function (Choose between Loss and Accuracy)
def load_model(num):
  if num == 0:
    model.load_weights(path_c + 'Loss_CP - ' + selected_model + fine_t + '.h5')
  else:
    model.load_weights(path_c + 'Acc_CP - ' + selected_model + fine_t + '.h5')

load_model(1)


In [ ]:
model.evaluate(x_val, y_val)


<a name="4.4"></a>
### &ensp; *4.4 Fine Tuning*

In [ ]:
# Unfreeze transfered model layers, besides batch normalization ones
for layer in model.layers[unfreeze_layers:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate*0.01),
              metrics=['acc'])


In [ ]:
history = model.fit(train,
                    validation_data = val,
                    epochs = fine_tune_epochs,
                    steps_per_epoch = len(x_train) // batch_size,
                    callbacks = [checkpoint_acc_ft, checkpoint_loss_ft, lr_reducer,lr_scheduler, early_stopping])


In [ ]:
# Changed to fine tuned
fine_t = ' FT'

# Loead weights that achieved the highest accuracy
load_model(1)


<a name="5"></a>
## 5. Predictions

<a name="5.1"></a>
### &ensp; *5.1 Test Images*

In [ ]:
# Read Test Images
test_name = []
x_test = []
for img in os.listdir(test_path):
  img_array = cv2.imread(os.path.join(test_path, img))
  res_array = cv2.resize(img_array, (img_size, img_size))
  test_name.append(img)
  x_test.append(res_array)

x_test = np.array(x_test).reshape(-1, img_size, img_size, 3)


<a name="5.2"></a>
### &ensp; *5.2 Export Predictions* 

In [ ]:
# predict
result =  model.predict(x_test)
softmax = model.predict(x_test, verbose=1)

# get predict label
predict_label = np.argmax(result,axis=-1)

# Export CSV
Submission = pd.DataFrame(list(zip(test_name, predict_label)), columns = [colx, coly])
Submission.to_csv(sub + selected_model + ' - (' + str(img_size) + ',' + str(epochs) + ',' + str(batch_size) + ',' + str(learning_rate) + ',' + str(fine_tune_epochs) + ',' + str(unfreeze_layers) + ')' + fine_t + '.csv', index = False)

Softmax = pd.DataFrame(softmax)
Softmax.to_csv(sub + 'Softmax - ' + selected_model + ' - (' + str(img_size) + ',' + str(epochs) + ',' + str(batch_size) + ',' + str(learning_rate) + ',' + str(fine_tune_epochs) + ',' + str(unfreeze_layers) + ')' + fine_t + '.csv', index = False)


<a name="5.3"></a>
### &ensp; *5.3 Voting Ensemble*

In [ ]:
# Soft Voting
soft = os.listdir(soft_path)

# Create an array of zeros, with 3 columns, for each class, and columns equal
# to the number of test images.
# Read the CSVs from the path and sums the softmaxes values of all CSVs
soft_predictions = np.zeros([len(test_name), num_cl])
for soft_csv in soft:
  predictions = pd.read_csv(soft_path+'/'+soft_csv, dtype=float)
  soft_predictions = np.sum([soft_predictions, predictions], axis=0)

# Returns the index of the biggest value by row. Which is equal to the class.
soft_final = np.argmax(soft_predictions, axis=1)

results = pd.DataFrame(list(zip(test_name, soft_final)),
                       columns = [colx, coly])
results.to_csv(soft_single, index=False)
